In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

# find data folder regardless of where notebook sits
candidate_dirs = [Path("../data"), Path("../../data"), Path("data")]
DATA_DIR = next((d for d in candidate_dirs if d.exists()), None)
if DATA_DIR is None:
    raise FileNotFoundError("Couldn't find a 'data' folder at ../data, ../../data, or ./data")

CLEAN = DATA_DIR / "cleaned_readmissions.csv"
if not CLEAN.exists():
    raise FileNotFoundError(f"Missing {CLEAN}. Make sure Step 4 saved the cleaned file.")

df = pd.read_csv(CLEAN, dtype=str)  # load as string, we’ll control types next
print("Loaded:", CLEAN.resolve(), "shape:", df.shape)
df.head(3)


Loaded: /Users/meghanakurapati/Documents/projects/healthcare-readmission-analytics/data/cleaned_readmissions.csv shape: (18510, 14)


,facility_name,facility_id,state,measure_id,score,predicted_readmission_rate,expected_readmission_rate,number_of_discharges,number_of_readmissions,start_date,end_date,footnote,risk_band,condition
0,SOUTHEAST HEALTH MEDICAL CENTER,010001,AL,READM-30-AMI-HRRP,0.9483,13.0146,13.7235,296.0,36.0,2020-07-01,2023-06-30,NaN,Low,Acute Myocardial Infarction
1,SOUTHEAST HEALTH MEDICAL CENTER,010001,AL,READM-30-CABG-HRRP,0.9509,9.6899,10.1898,151.0,13.0,2020-07-01,2023-06-30,NaN,Low,CABG
2,SOUTHEAST HEALTH MEDICAL CENTER,010001,AL,READM-30-HF-HRRP,1.0597,21.5645,20.3495,681.0,151.0,2020-07-01,2023-06-30,NaN,High,Heart Failure


In [3]:
# 1) normalize column names: "Facility Name" -> "facility_name"
def normalize(cols):
    return [c.strip().lower().replace(" ", "_").replace("/", "_").replace("-", "_") for c in cols]
df.columns = normalize(df.columns)

# 2) make column names unique (prevents to_numeric errors when duplicates exist)
def make_unique(cols):
    seen = {}
    out = []
    for c in cols:
        if c not in seen:
            seen[c] = 0
            out.append(c)
        else:
            seen[c] += 1
            out.append(f"{c}__{seen[c]}")
    return out
df.columns = make_unique(df.columns)

# 3) harmonize measure name and score
# some files have "measure_name" containing READM-30-... codes; rename to "measure_id" if present
if "measure_name" in df.columns and "measure_id" not in df.columns:
    df = df.rename(columns={"measure_name": "measure_id"})

# choose a metric column and call it `score`
metric_priority = ["excess_readmission_ratio", "predicted_readmission_rate", "score"]
metric_found = next((c for c in metric_priority if c in df.columns), None)
if metric_found is None:
    raise ValueError(f"No metric column found. Expected one of {metric_priority}.")
if metric_found != "score":
    df["score"] = df[metric_found]

# 4) safe numeric conversions (handles "Too Few to Report" -> NaN)
numeric_bases = {"score","number_of_discharges","number_of_readmissions",
                 "predicted_readmission_rate","expected_readmission_rate"}
for col in df.columns:
    base = col.split("__")[0]
    if base in numeric_bases:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# 5) parse dates (if present)
for col in df.columns:
    base = col.split("__")[0]
    if base in {"start_date","end_date"}:
        df[col] = pd.to_datetime(df[col], errors="coerce")

# 6) clean facility_id (keep leading zeros, strip Excel ".0")
for col in df.columns:
    if col.split("__")[0] == "facility_id":
        df[col] = (df[col].astype(str).str.replace(".0","", regex=False).str.zfill(6))

df.head(3)

,facility_name,facility_id,state,measure_id,score,predicted_readmission_rate,expected_readmission_rate,number_of_discharges,number_of_readmissions,start_date,end_date,footnote,risk_band,condition
0,SOUTHEAST HEALTH MEDICAL CENTER,010001,AL,READM-30-AMI-HRRP,13.0146,13.0146,13.7235,296.0,36.0,2020-07-01,2023-06-30,NaN,Low,Acute Myocardial Infarction
1,SOUTHEAST HEALTH MEDICAL CENTER,010001,AL,READM-30-CABG-HRRP,9.6899,9.6899,10.1898,151.0,13.0,2020-07-01,2023-06-30,NaN,Low,CABG
2,SOUTHEAST HEALTH MEDICAL CENTER,010001,AL,READM-30-HF-HRRP,21.5645,21.5645,20.3495,681.0,151.0,2020-07-01,2023-06-30,NaN,High,Heart Failure


In [4]:
# 1) condition label from READM-30 codes
measure_map = {
    "READM-30-AMI-HRRP": "Acute MI",
    "READM-30-HF-HRRP": "Heart Failure",
    "READM-30-PN-HRRP": "Pneumonia",
    "READM-30-COPD-HRRP": "COPD",
    "READM-30-CABG-HRRP": "CABG",
    "READM-30-HIP-KNEE-HRRP": "Hip/Knee"
}
if "measure_id" in df.columns:
    df["condition"] = df["measure_id"].map(measure_map).fillna(df["measure_id"])
else:
    df["condition"] = "All Measures"

# 2) engineered metrics (only if inputs exist)
if "predicted_readmission_rate" in df.columns and "expected_readmission_rate" in df.columns:
    df["delta_pred_exp"] = df["predicted_readmission_rate"] - df["expected_readmission_rate"]
    df["ratio_pred_exp"] = df["predicted_readmission_rate"] / df["expected_readmission_rate"]

if "number_of_readmissions" in df.columns and "number_of_discharges" in df.columns:
    df["readmit_rate_calc"] = df["number_of_readmissions"] / df["number_of_discharges"]

# 3) observation window length (days and years) if dates exist
if {"start_date","end_date"}.issubset(df.columns):
    df["period_days"] = (df["end_date"] - df["start_date"]).dt.days
    df["period_years"] = df["period_days"] / 365.25

df.head(5)

,facility_name,facility_id,state,measure_id,score,predicted_readmission_rate,expected_readmission_rate,number_of_discharges,number_of_readmissions,start_date,end_date,footnote,risk_band,condition,delta_pred_exp,ratio_pred_exp,readmit_rate_calc,period_days,period_years
0,SOUTHEAST HEALTH MEDICAL CENTER,010001,AL,READM-30-AMI-HRRP,13.0146,13.0146,13.7235,296.0,36.0,2020-07-01,2023-06-30,NaN,Low,Acute MI,-0.7089,0.948344,0.121622,1094,2.995209
1,SOUTHEAST HEALTH MEDICAL CENTER,010001,AL,READM-30-CABG-HRRP,9.6899,9.6899,10.1898,151.0,13.0,2020-07-01,2023-06-30,NaN,Low,CABG,-0.4999,0.950941,0.086093,1094,2.995209
2,SOUTHEAST HEALTH MEDICAL CENTER,010001,AL,READM-30-HF-HRRP,21.5645,21.5645,20.3495,681.0,151.0,2020-07-01,2023-06-30,NaN,High,Heart Failure,1.2150,1.059707,0.221733,1094,2.995209
3,SOUTHEAST HEALTH MEDICAL CENTER,010001,AL,READM-30-HIP-KNEE-HRRP,4.2680,4.2680,4.4211,NaN,NaN,2020-07-01,2023-06-30,NaN,Moderate,Hip/Knee,-0.1531,0.965371,NaN,1094,2.995209
4,SOUTHEAST HEALTH MEDICAL CENTER,010001,AL,READM-30-PN-HRRP,16.1137,16.1137,16.5863,490.0,77.0,2020-07-01,2023-06-30,NaN,Moderate,Pneumonia,-0.4726,0.971507,0.157143,1094,2.995209


In [5]:
# 1) volume bands by discharges
if "number_of_discharges" in df.columns and df["number_of_discharges"].notna().sum() >= 4:
    q = df["number_of_discharges"].quantile([0.33, 0.66]).tolist()
    def vol_band(x):
        if pd.isna(x): return np.nan
        if x <= q[0]: return "Low Volume"
        if x <= q[1]: return "Medium Volume"
        return "High Volume"
    df["volume_band"] = df["number_of_discharges"].apply(vol_band)

# 2) fixed threshold risk flags (tune thresholds if needed)
# HRRP-like: >1 means worse than expected; we flag high at >=1.05, low at <=0.95
df["high_risk"] = (df["score"] >= 1.05).astype("Int64")
df["low_risk"]  = (df["score"] <= 0.95).astype("Int64")

# 3) quantile-based risk_band (robust categorization)
if df["score"].notna().sum() >= 4:
    df["risk_band"] = pd.qcut(df["score"], q=4, labels=["Low","Moderate","Elevated","High"])

# 4) z-scores by condition (how far a hospital is from peers in that condition)
if {"condition","score"}.issubset(df.columns):
    def z_by_group(s):
        m, sd = s.mean(), s.std(ddof=0)
        return (s - m) / sd if sd and not np.isclose(sd, 0) else pd.Series([0]*len(s), index=s.index)
    df["score_z_condition"] = df.groupby("condition")["score"].transform(z_by_group)

# 5) z-scores by state (optional)
if {"state","score"}.issubset(df.columns):
    def z_by_group2(s):
        m, sd = s.mean(), s.std(ddof=0)
        return (s - m) / sd if sd and not np.isclose(sd, 0) else pd.Series([0]*len(s), index=s.index)
    df["score_z_state"] = df.groupby("state")["score"].transform(z_by_group2)

df.head(10)

,facility_name,facility_id,state,measure_id,score,predicted_readmission_rate,expected_readmission_rate,number_of_discharges,number_of_readmissions,start_date,...,delta_pred_exp,ratio_pred_exp,readmit_rate_calc,period_days,period_years,volume_band,high_risk,low_risk,score_z_condition,score_z_state
0,SOUTHEAST HEALTH MEDICAL CENTER,010001,AL,READM-30-AMI-HRRP,13.0146,13.0146,13.7235,296.0,36.0,2020-07-01,...,-0.7089,0.948344,0.121622,1094,2.995209,High Volume,1,0,-0.268145,-0.360718
1,SOUTHEAST HEALTH MEDICAL CENTER,010001,AL,READM-30-CABG-HRRP,9.6899,9.6899,10.1898,151.0,13.0,2020-07-01,...,-0.4999,0.950941,0.086093,1094,2.995209,Medium Volume,1,0,-0.671246,-1.083823
2,SOUTHEAST HEALTH MEDICAL CENTER,010001,AL,READM-30-HF-HRRP,21.5645,21.5645,20.3495,681.0,151.0,2020-07-01,...,1.2150,1.059707,0.221733,1094,2.995209,High Volume,1,0,1.072423,1.498841
3,SOUTHEAST HEALTH MEDICAL CENTER,010001,AL,READM-30-HIP-KNEE-HRRP,4.2680,4.2680,4.4211,NaN,NaN,2020-07-01,...,-0.1531,0.965371,NaN,1094,2.995209,NaN,1,0,-0.659029,-2.263058
4,SOUTHEAST HEALTH MEDICAL CENTER,010001,AL,READM-30-PN-HRRP,16.1137,16.1137,16.5863,490.0,77.0,2020-07-01,...,-0.4726,0.971507,0.157143,1094,2.995209,High Volume,1,0,0.072625,0.313320
5,SOUTHEAST HEALTH MEDICAL CENTER,010001,AL,READM-30-COPD-HRRP,15.4544,15.4544,16.5637,130.0,16.0,2020-07-01,...,-1.1093,0.933028,0.123077,1094,2.995209,Low Volume,1,0,-1.339259,0.169926
6,MARSHALL MEDICAL CENTERS,010005,AL,READM-30-CABG-HRRP,NaN,NaN,NaN,NaN,NaN,2020-07-01,...,NaN,NaN,NaN,1094,2.995209,NaN,0,0,NaN,NaN
7,MARSHALL MEDICAL CENTERS,010005,AL,READM-30-HIP-KNEE-HRRP,4.2346,4.2346,4.9836,NaN,NaN,2020-07-01,...,-0.7490,0.849707,NaN,1094,2.995209,NaN,1,0,-0.687438,-2.270322
8,MARSHALL MEDICAL CENTERS,010005,AL,READM-30-HF-HRRP,20.1511,20.1511,20.2835,176.0,35.0,2020-07-01,...,-0.1324,0.993473,0.198864,1094,2.995209,Medium Volume,1,0,0.350667,1.191434
9,MARSHALL MEDICAL CENTERS,010005,AL,READM-30-PN-HRRP,13.3621,13.3621,15.7296,305.0,30.0,2020-07-01,...,-2.3675,0.849488,0.098361,1094,2.995209,High Volume,1,0,-1.351901,-0.285138


In [6]:
# nulls
nulls = df.isna().mean().sort_values(ascending=False).head(10)
display(nulls)

# drop exact duplicate rows if any
before = len(df)
df = df.drop_duplicates()
after = len(df)
print(f"Dropped {before - after} duplicate rows.")

footnote                  0.644354
readmit_rate_calc         0.561264
number_of_readmissions    0.561264
volume_band               0.549433
number_of_discharges      0.549433
risk_band                 0.355646
score_z_condition         0.355646
ratio_pred_exp            0.355646
delta_pred_exp            0.355646
score_z_state             0.355646
dtype: float64

Dropped 0 duplicate rows.


In [7]:
keep = [
    "facility_name","facility_id","state","condition","measure_id",
    "score","predicted_readmission_rate","expected_readmission_rate",
    "number_of_discharges","number_of_readmissions",
    "readmit_rate_calc","delta_pred_exp","ratio_pred_exp",
    "risk_band","high_risk","low_risk","volume_band",
    "score_z_condition","score_z_state",
    "start_date","end_date","period_days","period_years","footnote"
]

cols = [c for c in keep if c in df.columns]
df_out = df[cols].copy()
print("Output shape:", df_out.shape)
df_out.head(5)

Output shape: (18510, 24)


,facility_name,facility_id,state,condition,measure_id,score,predicted_readmission_rate,expected_readmission_rate,number_of_discharges,number_of_readmissions,...,high_risk,low_risk,volume_band,score_z_condition,score_z_state,start_date,end_date,period_days,period_years,footnote
0,SOUTHEAST HEALTH MEDICAL CENTER,010001,AL,Acute MI,READM-30-AMI-HRRP,13.0146,13.0146,13.7235,296.0,36.0,...,1,0,High Volume,-0.268145,-0.360718,2020-07-01,2023-06-30,1094,2.995209,NaN
1,SOUTHEAST HEALTH MEDICAL CENTER,010001,AL,CABG,READM-30-CABG-HRRP,9.6899,9.6899,10.1898,151.0,13.0,...,1,0,Medium Volume,-0.671246,-1.083823,2020-07-01,2023-06-30,1094,2.995209,NaN
2,SOUTHEAST HEALTH MEDICAL CENTER,010001,AL,Heart Failure,READM-30-HF-HRRP,21.5645,21.5645,20.3495,681.0,151.0,...,1,0,High Volume,1.072423,1.498841,2020-07-01,2023-06-30,1094,2.995209,NaN
3,SOUTHEAST HEALTH MEDICAL CENTER,010001,AL,Hip/Knee,READM-30-HIP-KNEE-HRRP,4.2680,4.2680,4.4211,NaN,NaN,...,1,0,NaN,-0.659029,-2.263058,2020-07-01,2023-06-30,1094,2.995209,NaN
4,SOUTHEAST HEALTH MEDICAL CENTER,010001,AL,Pneumonia,READM-30-PN-HRRP,16.1137,16.1137,16.5863,490.0,77.0,...,1,0,High Volume,0.072625,0.313320,2020-07-01,2023-06-30,1094,2.995209,NaN


In [8]:
# save model-ready table
MODEL_CSV = DATA_DIR / "model_input.csv"
MODEL_PARQUET = DATA_DIR / "model_input.parquet"
df_out.to_csv(MODEL_CSV, index=False)
try:
    df_out.to_parquet(MODEL_PARQUET, index=False)
except Exception as e:
    print("Parquet save skipped:", e)

print("Saved:", MODEL_CSV.resolve())
if MODEL_PARQUET.exists():
    print("Saved:", MODEL_PARQUET.resolve())

# write a small data dictionary
DICT = DATA_DIR / "data_dictionary.md"
dictionary = """
# Data Dictionary — model_input

| Column                 | Type      | Description |
|------------------------|-----------|-------------|
| facility_name          | string    | Hospital name |
| facility_id            | string    | 6-digit hospital ID (zero-padded) |
| state                  | string    | US state code |
| condition              | string    | Friendly label mapped from measure codes |
| measure_id             | string    | READM-30-* HRRP measure code |
| score                  | float     | Main metric (Excess Readmission Ratio or Predicted Rate) |
| predicted_readmission_rate | float | CMS predicted readmission rate (if available) |
| expected_readmission_rate  | float | CMS expected readmission rate (if available) |
| number_of_discharges   | float     | Eligible discharges |
| number_of_readmissions | float     | Observed readmissions |
| readmit_rate_calc      | float     | Computed readmissions/discharges |
| delta_pred_exp         | float     | Predicted - Expected readmission rate |
| ratio_pred_exp         | float     | Predicted / Expected readmission rate |
| risk_band              | category  | Quartile band of `score` (Low→High) |
| high_risk              | 0/1       | 1 if score ≥ 1.05 |
| low_risk               | 0/1       | 1 if score ≤ 0.95 |
| volume_band            | category  | Discharge volume band (Low/Medium/High) |
| score_z_condition      | float     | z-score of `score` within each condition |
| score_z_state          | float     | z-score of `score` within each state |
| start_date             | date      | Period start (if present) |
| end_date               | date      | Period end (if present) |
| period_days            | float     | end_date - start_date in days |
| period_years           | float     | period_days / 365.25 |
"""
DICT.write_text(dictionary.strip())
print("Wrote dictionary:", DICT.resolve())

Saved: /Users/meghanakurapati/Documents/projects/healthcare-readmission-analytics/data/model_input.csv
Saved: /Users/meghanakurapati/Documents/projects/healthcare-readmission-analytics/data/model_input.parquet
Wrote dictionary: /Users/meghanakurapati/Documents/projects/healthcare-readmission-analytics/data/data_dictionary.md


In [9]:
pd.read_csv(MODEL_CSV, nrows=5)

,facility_name,facility_id,state,condition,measure_id,score,predicted_readmission_rate,expected_readmission_rate,number_of_discharges,number_of_readmissions,...,high_risk,low_risk,volume_band,score_z_condition,score_z_state,start_date,end_date,period_days,period_years,footnote
0,SOUTHEAST HEALTH MEDICAL CENTER,10001,AL,Acute MI,READM-30-AMI-HRRP,13.0146,13.0146,13.7235,296.0,36.0,...,1,0,High Volume,-0.268145,-0.360718,2020-07-01,2023-06-30,1094,2.995209,NaN
1,SOUTHEAST HEALTH MEDICAL CENTER,10001,AL,CABG,READM-30-CABG-HRRP,9.6899,9.6899,10.1898,151.0,13.0,...,1,0,Medium Volume,-0.671246,-1.083823,2020-07-01,2023-06-30,1094,2.995209,NaN
2,SOUTHEAST HEALTH MEDICAL CENTER,10001,AL,Heart Failure,READM-30-HF-HRRP,21.5645,21.5645,20.3495,681.0,151.0,...,1,0,High Volume,1.072423,1.498841,2020-07-01,2023-06-30,1094,2.995209,NaN
3,SOUTHEAST HEALTH MEDICAL CENTER,10001,AL,Hip/Knee,READM-30-HIP-KNEE-HRRP,4.2680,4.2680,4.4211,NaN,NaN,...,1,0,NaN,-0.659029,-2.263058,2020-07-01,2023-06-30,1094,2.995209,NaN
4,SOUTHEAST HEALTH MEDICAL CENTER,10001,AL,Pneumonia,READM-30-PN-HRRP,16.1137,16.1137,16.5863,490.0,77.0,...,1,0,High Volume,0.072625,0.313320,2020-07-01,2023-06-30,1094,2.995209,NaN
